In [20]:
import pandas as pd

file_path = "data_diem_chuan_nam_2024.csv"
df = pd.read_csv(file_path)

print(df.head())

print(df.info())

            Nhóm ngành    Ngành            Trường đào tạo    Mã ngành  \
0  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân     7340301   
1  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân        EP04   
2  Kế toán - Kiểm toán  Kế toán  Đại Học Bách Khoa Hà Nội         EM4   
3  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính     7340301   
4  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính  7340301C21   

                                        Tên ngành          Tổ hợp môn  \
0                                         Kế toán  A00; A01; D01; D07   
1  Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB)  A00; A01; D01; D07   
2                                         Kế toán       A00; A01; D01   
3                                         Kế toán  A00; A01; D01; D07   
4     Kế toán doanh nghiệp (theo định hướng ACCA)       A01; D01; D07   

   Điểm chuẩn 2024                                Ghi chú  
0            27.29                                    NaN  
1 

In [21]:
print(df.isnull().sum())

Nhóm ngành            0
Ngành                 0
Trường đào tạo        0
Mã ngành              0
Tên ngành             0
Tổ hợp môn           37
Điểm chuẩn 2024       0
Ghi chú            4426
dtype: int64


In [22]:
df["Tổ hợp môn"].fillna("Không rõ", inplace=True)

C:\Users\PC\AppData\Local\Temp\ipykernel_2668\610147126.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Tổ hợp môn"].fillna("Không rõ", inplace=True)


In [23]:
print(df.isnull().sum())
print(df.head())

Nhóm ngành            0
Ngành                 0
Trường đào tạo        0
Mã ngành              0
Tên ngành             0
Tổ hợp môn            0
Điểm chuẩn 2024       0
Ghi chú            4426
dtype: int64
            Nhóm ngành    Ngành            Trường đào tạo    Mã ngành  \
0  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân     7340301   
1  Kế toán - Kiểm toán  Kế toán  Đại Học Kinh Tế Quốc Dân        EP04   
2  Kế toán - Kiểm toán  Kế toán  Đại Học Bách Khoa Hà Nội         EM4   
3  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính     7340301   
4  Kế toán - Kiểm toán  Kế toán        Học Viện Tài chính  7340301C21   

                                        Tên ngành          Tổ hợp môn  \
0                                         Kế toán  A00; A01; D01; D07   
1  Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB)  A00; A01; D01; D07   
2                                         Kế toán       A00; A01; D01   
3                                         Kế toán  A00; A01; D01

In [24]:
cleaned_file_path = "data_diem_chuan_cleaned.csv"
df.to_csv(cleaned_file_path, index=False)
print(f"Dữ liệu đã lưu vào {cleaned_file_path}")

Dữ liệu đã lưu vào data_diem_chuan_cleaned.csv


In [25]:
import json

train_data_path = "train_data.jsonl"
with open(train_data_path, "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        print(json.loads(line))

{'prompt': 'Ngành Kế toán tại Đại Học Kinh Tế Quốc Dân lấy bao nhiêu điểm năm 2024?', 'completion': 'Năm 2024, ngành Kế toán tại Đại Học Kinh Tế Quốc Dân có điểm chuẩn là 27.29.'}
{'prompt': 'Em muốn học ngành Kế toán thì cần thi tổ hợp môn nào?', 'completion': 'Ngành Kế toán xét tuyển theo các tổ hợp môn: A00; A01; D01; D07.'}
{'prompt': 'Nhóm ngành Kế toán - Kiểm toán bao gồm những ngành nào?', 'completion': 'Nhóm ngành Kế toán - Kiểm toán bao gồm ngành Kế toán và nhiều ngành khác.'}
{'prompt': 'Mã ngành của Kế toán tại Đại Học Kinh Tế Quốc Dân là gì?', 'completion': 'Mã ngành của Kế toán tại Đại Học Kinh Tế Quốc Dân là 7340301.'}
{'prompt': 'Ngành Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB) tại Đại Học Kinh Tế Quốc Dân lấy bao nhiêu điểm năm 2024?', 'completion': 'Năm 2024, ngành Kế toán tích hợp chứng chỉ quốc tế UCAEW CFAB) tại Đại Học Kinh Tế Quốc Dân có điểm chuẩn là 27.2.'}


In [26]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import json

torch.cuda.empty_cache()

model_name = "VietAI/gpt-neo-1.3B-vietnamese-news"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

train_data_path = "train_data.jsonl"
data_list = []

with open(train_data_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        prompt = item["prompt"]
        completion = item["completion"]
        data_list.append({"text": f"Người dùng: {prompt}\nChatbot: {completion}"})

raw_dataset = Dataset.from_list(data_list)


def preprocess_function(examples):
    inputs = tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors=None
    )

    inputs["labels"] = inputs["input_ids"].copy()
    return inputs

In [27]:
tokenized_dataset = raw_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["text"]
)

Map: 100%|██████████| 276712/276712 [00:23<00:00, 11724.93 examples/s]


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16
)

peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["c_attn", "c_proj", "c_fc"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

training_args = TrainingArguments(
    output_dir="./fine_tuned_gpt2_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    learning_rate=2e-4,
    lr_scheduler_type="cosine",
    save_strategy="steps",
    save_steps=50,
    logging_steps=10,
    max_steps=100,
    fp16=True,
    remove_unused_columns=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

trainer.train()

peft_model_id = "./fine_tuned_gpt2_model_final"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

trainable params: 3,932,160 || all params: 1,339,461,632 || trainable%: 0.2936


C:\Users\PC\AppData\Local\Temp\ipykernel_2668\1468519940.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
10,2.717300


In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Testing model...")
model.eval()


def generate_response(prompt, model, tokenizer, max_length=150):
    input_text = f"Người dùng: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        pad_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=pad_token_id
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Chatbot:" in response:
        response = response.split("Chatbot:")[1].strip()
    else:
        response = response.strip()
    return response


test_prompt = "Tôi giỏi Theo dõi tin quốc tế, có thể học ngành nào?"
response = generate_response(test_prompt, model, tokenizer)
print(f"Test Prompt: {test_prompt}")
print(f"Response: {response}")

Testing model...
Test Prompt: Tôi giỏi Theo dõi tin quốc tế, có thể học ngành nào?
Response: Với sở thích Theo dõi tin quốc tế, bạn có thể học ngành Công nghệ thông tin tại Trường Đại Học Công Nghệ TPHCM nếu đam mê. Điểm chuẩn ngành này năm 2024 là 24.0.


In [ ]:
import torch
import faiss
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Testing model...")
model.eval()

data_path = "data_diem_chuan_cleaned.csv"
df = pd.read_csv(data_path)

tfidf = TfidfVectorizer()
df["vectorized"] = list(tfidf.fit_transform(df["Tên ngành"].astype(str)).toarray())

d = len(df["vectorized"].iloc[0])
index = faiss.IndexFlatL2(d)
index.add(np.array(df["vectorized"].tolist(), dtype='float32'))


def search_major(query, top_k=5):
    query_vec = tfidf.transform([query]).toarray().astype('float32')
    _, idxs = index.search(query_vec, top_k)
    return df.iloc[idxs[0]][["Trường đào tạo", "Tên ngành", "Điểm chuẩn 2024"]]


model_name = "fine_tuned_gpt2_model_final"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)


def generate_response(prompt, model, tokenizer, max_length=150):
    input_text = f"Người dùng: {prompt}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)

    with torch.no_grad():
        pad_token_id = tokenizer.eos_token_id if tokenizer.eos_token_id is not None else tokenizer.pad_token_id
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=pad_token_id
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Chatbot:" in response:
        response = response.split("Chatbot:")[1].strip()
    else:
        response = response.strip()
    return response


if __name__ == "__main__":
    user_input = "công nghệ thông tin"  # input("Nhập sở thích/ngành bạn muốn tìm: ")
    results = search_major(user_input)
    print("\nGợi ý ngành học phù hợp:")
    print(results)

    prompt = f"Tôi thích {user_input}, có thể học ngành nào?"
    response = generate_response(prompt, model, tokenizer)
    print("\nChatbot tư vấn:")
    print(response)

Testing model...

Gợi ý ngành học phù hợp:
                               Trường đào tạo            Tên ngành  \
1017                 Đại Học Kinh Tế Quốc Dân  Công nghệ thông tin   
1021                       Học Viện Ngân Hàng  Công nghệ thông tin   
1022  Học Viện Công Nghệ Bưu Chính Viễn Thông  Công nghệ thông tin   
1023  Học Viện Công Nghệ Bưu Chính Viễn Thông  Công nghệ thông tin   
1025             Trường Đại Học Sư Phạm TPHCM  Công nghệ thông tin   

      Điểm chuẩn 2024  
1017            35.17  
1021            25.80  
1022            26.40  
1023            25.43  
1025            23.05  


c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\transformers\generation\configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\transformers\generation\configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
c:\Users\PC\anaconda3\envs\orb-slam-py\lib\site-packages\transformers\generation\utils.py:2184: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)